In [1]:
import numpy as np
import glob
import os
import pylab
import scipy.misc
import skimage
from skimage import io
from scipy import ndimage
import matplotlib
import matplotlib.pyplot as plt

In [2]:
def get_padded_slices(slices, pad, label):
    
    new_slices = []
    for i,s in enumerate(slices):
            
        start = s.start
        stop = s.stop
                
        if i == 3:
            
            # my edits to prevent slicing of the channels
            new_start = 0
            new_stop = 3
            # end of my edits
            
            #fairly's original code below
            #new_slices.append(s)
            
            # my edited code
            new_slices.append(slice(new_start, new_stop))
            continue
            
        if start-pad < 0:
            new_start = 0
        else:
            new_start = start-pad
            
        if stop+pad > label.shape[i]:
            new_stop = label.shape[i]
        else:
            new_stop = stop+pad
            
        new_slices.append(slice(new_start, new_stop))
        
    return tuple(new_slices)

In [3]:
def label_mask(mask):
    
    label, n = ndimage.label(mask)
    return label, n

In [4]:
def load_mask(mask_file):
    
    mask = io.imread(mask_file)
    return mask

In [5]:
def load_original_image(image_file):
    
    original_image = io.imread(image_file)
    return original_image

In [6]:
def crop_image(original_image, new_slices):
    
    cropped_image = original_image[new_slices]
    return cropped_image

In [7]:
def save_cropped_image(image_file):
    
    skimage.io.imsave((image_file + str(i)))

In [8]:
def get_slices(label):
    
    slices = ndimage.find_objects(label)
    return slices

In [9]:
def extract_inclusions(mask_file, image_file, pad):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    
    for i in range (0, n):
        
        new_slices = get_padded_slices(slices[i], pad, label)
        cropped_image = original_image[new_slices]
        skimage.io.imsave((image_file[:-4] + ' cropped ' + str(i) + '.tif'), cropped_image)
        
        
        
        

In [10]:
def get_64x64x64_cubes(slices, label):
    
    new_slices = []
    for i,s in enumerate(slices):
            
        start = s.start
        stop = s.stop
                
        if i == 3:
            
            # my edits to prevent slicing of the channels
            new_start = 0
            new_stop = 3
            # end of my edits
            
            #fairly's original code below
            #new_slices.append(s)
            
            # my edited code
            new_slices.append(slice(new_start, new_stop))
            continue
            
        if round((start + stop)/2)-32 < 0:
            break
        else:
            new_start = round((start + stop)/2) - 32
            
        if round((start + stop)/2)+32 > label.shape[i]:
            break
        else:
            new_stop = round((start + stop)/2)+32
            
        new_slices.append(slice(new_start, new_stop))
        
    return tuple(new_slices)

In [11]:
def extract_cubed_inclusions(mask_file, image_file):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    counter = 0
    for i in range (0, n):
        
        new_slices = get_64x64x64_cubes(slices[i], label)
        
        if len(new_slices) < 4:
            continue
        
        else:
            cropped_image = original_image[new_slices]
            skimage.io.imsave((image_file[:-4] + ' cropped ' + str(counter) + '.tif'), cropped_image)
            counter+=1
    return slices, counter

In [12]:
def get_padded_slices_v2(slices, pad, label):
    
    new_slices = []
    for i,s in enumerate(slices):
            
        start = s.start
        stop = s.stop
                
        if i == 3:
            
            # my edits to prevent slicing of the channels
            new_start = 0
            new_stop = 3
            # end of my edits
            
            #fairly's original code below
            #new_slices.append(s)
            
            # my edited code
            new_slices.append(slice(new_start, new_stop))
            continue
            
        if start-pad < 0:
            new_start = 0
        else:
            new_start = start-pad
            
        if stop+pad > label.shape[i]:
            new_stop = label.shape[i]
        else:
            new_stop = stop+pad
        
        if new_stop-new_start > 64:
            break
        else:
            new_slices.append(slice(new_start, new_stop))
        
    return tuple(new_slices)

In [13]:
def pad_to_64(cropped_shape, cropped_image):
    for i in range(0,3):
        if cropped_shape[i] % 2 == 1:
            if i == 0:
                a = int((64-cropped_shape[i] + 1)/2)
                b = a-1
            if i == 1:
                c = int((64-cropped_shape[i] + 1)/2)
                d = c-1
            if i == 2:
                e = int((64-cropped_shape[i] + 1)/2)
                f = e-1
        else:
            if i == 0:
                a = int((64-cropped_shape[i])/2)
                b = int((64-cropped_shape[i])/2)
            if i == 1:
                c = int((64-cropped_shape[i])/2)
                d = int((64-cropped_shape[i])/2)
            if i == 2:
                e = int((64-cropped_shape[i])/2)
                f = int((64-cropped_shape[i])/2)
                
    padded_cropped_image = np.pad(cropped_image,((a,b), (c,d), (e,f), (0, 0)), 'constant')
    return padded_cropped_image

In [14]:
def extract_padded_cropped_inclusions(mask_file, image_file, pad):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    counter = 0
    for i in range (0, n):
        
        new_slices = get_padded_slices_v2(slices[i], pad, label)
        
        if len(new_slices) < 4:
            continue
        
        else:
            cropped_image = original_image[new_slices]
            cropped_shape = cropped_image.shape
            padded_cropped_image = pad_to_64(cropped_shape, cropped_image)        
            skimage.io.imsave((image_file[:-4] + ' padded cropped ' + str(counter) + '.tif'), padded_cropped_image)
            counter+=1
    return slices, counter

In [15]:
def extract_inclusions(mask_file, image_file, pad):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    counter = 0
    for i in range (0, n):
        
        new_slices = get_padded_slices(slices[i], pad, label)
        cropped_image = original_image[new_slices]
        skimage.io.imsave((image_file[:-4] + ' cropped ' + str(i) + '.tif'), cropped_image)
        

In [16]:
cd /home/monuki/Documents/AI code/CropPadTest

/home/monuki/Documents/AI code/CropPadTest


In [17]:
!ls

UCI-43-18_C3.tif  UCI-43-18 raw.tif


In [18]:
extract_padded_cropped_inclusions('UCI-43-18_C3.tif', 'UCI-43-18 raw.tif', 10)

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 4.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 82.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 83.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 84.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 85.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 86.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 164.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 165.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 166.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 167.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 168.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 244.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 245.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 246.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 247.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 248.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 322.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 323.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 324.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 325.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 326.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

...

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 579.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 580.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 581.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 582.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 583.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 623.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 624.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 625.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 626.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 627.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 706.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 707.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 708.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 709.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 710.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 751.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 752.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


([(slice(4, 41, None),
   slice(429, 470, None),
   slice(829, 870, None),
   slice(1, 2, None)),
  (slice(5, 19, None),
   slice(195, 208, None),
   slice(941, 952, None),
   slice(1, 2, None)),
  (slice(5, 35, None),
   slice(365, 402, None),
   slice(851, 869, None),
   slice(1, 2, None)),
  (slice(5, 22, None),
   slice(470, 499, None),
   slice(865, 885, None),
   slice(1, 2, None)),
  (slice(5, 35, None),
   slice(842, 859, None),
   slice(753, 780, None),
   slice(1, 2, None)),
  (slice(5, 25, None),
   slice(894, 922, None),
   slice(606, 637, None),
   slice(1, 2, None)),
  (slice(6, 30, None),
   slice(123, 143, None),
   slice(672, 685, None),
   slice(1, 2, None)),
  (slice(6, 34, None),
   slice(197, 213, None),
   slice(891, 930, None),
   slice(1, 2, None)),
  (slice(6, 21, None),
   slice(207, 222, None),
   slice(986, 1000, None),
   slice(1, 2, None)),
  (slice(6, 23, None),
   slice(229, 251, None),
   slice(924, 940, None),
   slice(1, 2, None)),
  (slice(6, 26, Non

In [22]:
file_list = glob.glob('*padded cropped*')

753

In [24]:
for i in range(0, len(file_list)):
    x = io.imread(file_list[i])
    y = np.max(x, axis=0)
    image_file = file_list[i]
    skimage.io.imsave(('MIP ' + image_file[:-4] + '.tif'), y)

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 261.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 664.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 619.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 316.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 56.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 735.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 32.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 323.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 266.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 88.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 482.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 112.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 750.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 198.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 670.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 458.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 532.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 200.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 424.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 429.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 600.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 222.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 608.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 274.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 207.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

...

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 139.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 319.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 485.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 68.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 675.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 744.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 436.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 481.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 484.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 606.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 631.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 176.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 747.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 51.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 478.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 468.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 719.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 247.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 216.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 369.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 397.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 150.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 671.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 324.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 195.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

In [54]:
cd /home/monuki/Documents/AI code/CropPadTestnoBuff


/home/monuki/Documents/AI code/CropPadTestnoBuff


In [56]:
!ls

UCI-43-18_C3.tif		      UCI-43-18 raw padded cropped 468.tif
UCI-43-18 raw padded cropped 0.tif    UCI-43-18 raw padded cropped 469.tif
UCI-43-18 raw padded cropped 100.tif  UCI-43-18 raw padded cropped 46.tif
UCI-43-18 raw padded cropped 101.tif  UCI-43-18 raw padded cropped 470.tif
UCI-43-18 raw padded cropped 102.tif  UCI-43-18 raw padded cropped 471.tif
UCI-43-18 raw padded cropped 103.tif  UCI-43-18 raw padded cropped 472.tif
UCI-43-18 raw padded cropped 104.tif  UCI-43-18 raw padded cropped 473.tif
UCI-43-18 raw padded cropped 105.tif  UCI-43-18 raw padded cropped 474.tif
UCI-43-18 raw padded cropped 106.tif  UCI-43-18 raw padded cropped 475.tif
UCI-43-18 raw padded cropped 107.tif  UCI-43-18 raw padded cropped 476.tif
UCI-43-18 raw padded cropped 108.tif  UCI-43-18 raw padded cropped 477.tif
UCI-43-18 raw padded cropped 109.tif  UCI-43-18 raw padded cropped 478.tif
UCI-43-18 raw padded cropped 10.tif   UCI-43-18 raw padded cropped 479.tif
UCI-43-18 raw padded cropped 110.tif  UC

In [30]:
slices, counter = extract_padded_cropped_inclusions('UCI-43-18_C3.tif', 'UCI-43-18 raw.tif', 0)

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 4.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 51.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 52.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 53.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 54.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 55.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 103.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 104.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 105.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 106.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 107.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 154.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 155.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 156.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 157.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 158.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 204.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 205.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 206.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 207.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 208.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

...

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 567.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 568.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 569.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 570.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 571.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 620.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 621.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 622.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 623.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 624.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 672.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 673.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 674.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 675.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 676.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 724.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 725.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 726.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 727.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 728.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 777.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 778.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 779.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 780.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 781.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

In [48]:
cd /home/monuki/Documents/AI code/CropPadTest5buff

/home/monuki/Documents/AI code/CropPadTest5buff


In [49]:
!ls

UCI-43-18_C3.tif		      UCI-43-18 raw padded cropped 461.tif
UCI-43-18 raw padded cropped 0.tif    UCI-43-18 raw padded cropped 462.tif
UCI-43-18 raw padded cropped 100.tif  UCI-43-18 raw padded cropped 463.tif
UCI-43-18 raw padded cropped 101.tif  UCI-43-18 raw padded cropped 464.tif
UCI-43-18 raw padded cropped 102.tif  UCI-43-18 raw padded cropped 465.tif
UCI-43-18 raw padded cropped 103.tif  UCI-43-18 raw padded cropped 466.tif
UCI-43-18 raw padded cropped 104.tif  UCI-43-18 raw padded cropped 467.tif
UCI-43-18 raw padded cropped 105.tif  UCI-43-18 raw padded cropped 468.tif
UCI-43-18 raw padded cropped 106.tif  UCI-43-18 raw padded cropped 469.tif
UCI-43-18 raw padded cropped 107.tif  UCI-43-18 raw padded cropped 46.tif
UCI-43-18 raw padded cropped 108.tif  UCI-43-18 raw padded cropped 470.tif
UCI-43-18 raw padded cropped 109.tif  UCI-43-18 raw padded cropped 471.tif
UCI-43-18 raw padded cropped 10.tif   UCI-43-18 raw padded cropped 472.tif
UCI-43-18 raw padded cropped 110.tif  UC

In [34]:
slices5, counter5 = extract_padded_cropped_inclusions('UCI-43-18_C3.tif', 'UCI-43-18 raw.tif', 5)

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 4.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 48.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 49.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 50.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 51.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 52.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 95.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 96.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 97.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 98.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 99.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 143.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 144.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 145.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 146.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 147.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 190.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 191.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 192.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 193.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 194.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

...

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 571.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 572.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 573.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 574.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 575.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 620.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 621.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 622.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 623.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 624.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 669.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 670.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 671.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 672.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 673.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 717.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 718.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 719.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 720.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 721.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 767.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 768.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 769.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 770.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: UCI-43-18 raw padded cropped 771.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/sk

In [35]:
file_list5buff = glob.glob('*padded cropped*')

In [50]:
counter5

802

In [51]:
len(file_list5buff)

802

In [53]:
for i in range(0, len(file_list5buff)):
    x = io.imread(file_list5buff[i])
    y = np.max(x, axis=0)
    image_file = file_list5buff[i]
    skimage.io.imsave(('MIP ' + image_file[:-4] + '.tif'), y)

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 261.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 664.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 619.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 316.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 56.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 32.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 323.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 768.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 266.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 88.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 416.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 693.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 590.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 83.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 489.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 276.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 537.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 121.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 518.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 752.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 440.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 499.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 308.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 549.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 514.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

...

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 271.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 522.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 492.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 510.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 515.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 105.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 326.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 79.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 139.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 319.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 757.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 586.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 711.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 646.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 727.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 683.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 19.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 64.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 194.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 371.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 275.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 154.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 65.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 505.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 221.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

In [41]:
file_listnobuff = glob.glob('*padded cropped*')

In [57]:
counter

815

In [58]:
len(file_listnobuff)

815

In [59]:
for i in range(0, len(file_listnobuff)):
    x = io.imread(file_listnobuff[i])
    y = np.max(x, axis=0)
    image_file = file_listnobuff[i]
    skimage.io.imsave(('MIP ' + image_file[:-4] + '.tif'), y)

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 261.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 664.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 619.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 316.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 56.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 306.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 304.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 239.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 69.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 427.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 11.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 205.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 289.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 251.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 185.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 750.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 198.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 670.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 401.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 61.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 409.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 433.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 400.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 778.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 77.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

...

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 234.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 105.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 326.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 79.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 139.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 453.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 745.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 757.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 586.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 711.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 27.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 714.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 660.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 45.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 90.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 508.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 74.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 451.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 533.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 136.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3

/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 403.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 48.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 759.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 570.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: MIP UCI-43-18 raw padded cropped 13.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/miniconda/envs/cnn/lib/python3.

In [45]:
counter5


802

In [46]:
cd

/home/monuki


In [47]:
!ls

Biondi inclusion extraction code.ipynb  jupyterhub-proxy.pid  temp/
Desktop/                                jupyterhub.sqlite     Templates/
Documents/                              Music/                Untitled.ipynb
Downloads/                              Pictures/             Videos/
examples.desktop                        Public/
jupyterhub_cookie_secret                scripts/
